## alexnet

In [8]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Custom Dataset for TIFF images
class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with all the TIFF images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.root_dir = root_dir
        self.transform = transform
        # List all TIFF files in the directory
        self.filenames = [f for f in os.listdir(root_dir) if f.lower().endswith(('.tif', '.tiff'))]

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        # Get the image file name and path
        img_name = self.filenames[idx]
        img_path = os.path.join(self.root_dir, img_name)
        # Open image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        # Label assignment based on filename:
        # Files containing 'K8' are labeled 0 and those containing 'K14' are labeled 1.
        if 'K8' in img_name:
            label = 0
        elif 'K14' in img_name:
            label = 1
        else:
            raise ValueError(f"Filename {img_name} does not contain a valid age identifier")
        
        return image, label

# Define data transforms (resize, tensor conversion, normalization)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalization for pretrained models
                         std=[0.229, 0.224, 0.225])
])

# Path to your folder with TIFF images
datafolder = "Old_Young_Comparison"

# Create the dataset
dataset = AgeDataset(root_dir=datafolder, transform=data_transforms)

# Train-test split (80% training, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load pretrained AlexNet and modify the final classifier layer for 2 classes
model = models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)  # Two output classes

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Function to calculate accuracy on a given DataLoader
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            # Get the predicted class (index of the highest score)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Training loop with validation
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()         # Clear gradients
        outputs = model(inputs)         # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()               # Backward pass
        optimizer.step()              # Update weights
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / train_size
    val_accuracy = calculate_accuracy(model, val_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/10, Loss: 0.7155, Validation Accuracy: 0.8214
Epoch 2/10, Loss: 0.4230, Validation Accuracy: 0.6071
Epoch 3/10, Loss: 0.5706, Validation Accuracy: 0.7500
Epoch 4/10, Loss: 0.4028, Validation Accuracy: 0.7500
Epoch 5/10, Loss: 0.4586, Validation Accuracy: 0.7857
Epoch 6/10, Loss: 0.2072, Validation Accuracy: 0.7857
Epoch 7/10, Loss: 0.1125, Validation Accuracy: 0.8929
Epoch 8/10, Loss: 0.0490, Validation Accuracy: 0.8571
Epoch 9/10, Loss: 0.0163, Validation Accuracy: 0.8929
Epoch 10/10, Loss: 0.0113, Validation Accuracy: 0.8571


In [13]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Custom Dataset for TIFF images
class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with all the TIFF images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.root_dir = root_dir
        self.transform = transform
        # List all TIFF files that contain either 'K8' or 'K14'
        self.filenames = [f for f in os.listdir(root_dir) 
                          if f.lower().endswith(('.tif', '.tiff')) and ('K14' in f)]
        
        # Build label mapping using the first letter of the file name
        # For example, if files start with 'O' or 'Y', then map them to numeric labels.
        unique_labels = sorted(set([f[0] for f in self.filenames]))
        self.label_map = {label: idx for idx, label in enumerate(unique_labels)}
        print(f"Label mapping: {self.label_map}")
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        # Get the image file name and full path
        img_name = self.filenames[idx]
        img_path = os.path.join(self.root_dir, img_name)
        # Open the image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        # Use the first letter of the file name as its age group label
        label_char = img_name[0]
        if label_char not in self.label_map:
            raise ValueError(f"Filename {img_name} has an invalid label character: {label_char}")
        label = self.label_map[label_char]
        return image, label

# Define data transforms (resize, tensor conversion, normalization)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalization for pretrained models
                         std=[0.229, 0.224, 0.225])
])

# Path to your folder with TIFF images
datafolder = "Old_Young_Comparison"

# Function to calculate accuracy on a given DataLoader
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            # Get the predicted class (index of the highest score)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

# Device configuration (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Experiment parameters
num_experiments = 50
num_epochs = 5
batch_size = 16

final_accuracies = []

for exp in range(num_experiments):
    print(f"\nStarting Experiment {exp+1}/{num_experiments}")
    
    # Create the dataset (filtered to only include files with 'K8' or 'K14')
    dataset = AgeDataset(root_dir=datafolder, transform=data_transforms)
    
    # Train-test split (80% training, 20% validation)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    # Create DataLoaders for training and validation sets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Load pretrained AlexNet and modify the final classifier layer
    model = models.alexnet(pretrained=True)
    num_ftrs = model.classifier[6].in_features
    # Set output classes to match the number of unique labels in the dataset
    model.classifier[6] = nn.Linear(num_ftrs, len(dataset.label_map))
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Training loop with validation
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()         # Clear gradients
            outputs = model(inputs)         # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()               # Backward pass
            optimizer.step()              # Update weights
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / train_size
        val_accuracy = calculate_accuracy(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    # Record the final epoch's validation accuracy for this experiment
    final_val_acc = calculate_accuracy(model, val_loader, device)
    final_accuracies.append(final_val_acc)
    print(f"Final Validation Accuracy for Experiment {exp+1}: {final_val_acc:.4f}")

# Compute and print the average final validation accuracy over all experiments
average_accuracy = sum(final_accuracies) / len(final_accuracies)
print(f"\nAverage Final Validation Accuracy over {num_experiments} experiments: {average_accuracy:.4f}")



Starting Experiment 1/50
Label mapping: {'O': 0, 'Y': 1}
Epoch 1/5, Loss: 1.4158, Validation Accuracy: 0.6429
Epoch 2/5, Loss: 0.7233, Validation Accuracy: 0.7143
Epoch 3/5, Loss: 0.5683, Validation Accuracy: 0.7143
Epoch 4/5, Loss: 0.5601, Validation Accuracy: 0.7143
Epoch 5/5, Loss: 0.5286, Validation Accuracy: 0.7143
Final Validation Accuracy for Experiment 1: 0.7143

Starting Experiment 2/50
Label mapping: {'O': 0, 'Y': 1}
Epoch 1/5, Loss: 0.9647, Validation Accuracy: 0.5714
Epoch 2/5, Loss: 0.5750, Validation Accuracy: 0.5714
Epoch 3/5, Loss: 0.5394, Validation Accuracy: 0.5714
Epoch 4/5, Loss: 0.5923, Validation Accuracy: 0.5000
Epoch 5/5, Loss: 0.5245, Validation Accuracy: 0.5714
Final Validation Accuracy for Experiment 2: 0.5714

Starting Experiment 3/50
Label mapping: {'O': 0, 'Y': 1}
Epoch 1/5, Loss: 1.9623, Validation Accuracy: 0.2857
Epoch 2/5, Loss: 0.7128, Validation Accuracy: 0.7143
Epoch 3/5, Loss: 0.6554, Validation Accuracy: 0.7143
Epoch 4/5, Loss: 0.6700, Validation

In [20]:
import os
import time
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Custom Dataset for TIFF images
class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with all the TIFF images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.root_dir = root_dir
        self.transform = transform
        # List all TIFF files that contain 'K14'
        self.filenames = [f for f in os.listdir(root_dir) 
                          if f.lower().endswith(('.tif', '.tiff')) and ('K14' in f)]
        
        # Build label mapping using the first letter of the file name.
        # For example, if files start with 'O' or 'Y', then map them to numeric labels.
        unique_labels = sorted(set([f[0] for f in self.filenames]))
        self.label_map = {label: idx for idx, label in enumerate(unique_labels)}
        print(f"Label mapping: {self.label_map}")
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        # Get image filename and full path
        img_name = self.filenames[idx]
        img_path = os.path.join(self.root_dir, img_name)
        # Open image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        # Use the first letter of the file name as its age group label
        label_char = img_name[0]
        if label_char not in self.label_map:
            raise ValueError(f"Filename {img_name} has an invalid label character: {label_char}")
        label = self.label_map[label_char]
        return image, label

# Define data transforms (resize, tensor conversion, normalization)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (expected by most models)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Standard ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

# Path to your folder with TIFF images
datafolder = "Old_Young_Comparison"

# Function to calculate accuracy on a given DataLoader
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            # Get the predicted class (index of the highest score)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Device configuration (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dictionary with model configurations.
# Each entry contains:
# - a constructor function for the pretrained model, and
# - a modification lambda that replaces the final classifier head with one matching the dataset.
model_configs = {
    'ResNet50': {
        'constructor': models.resnet50,
        'modify': lambda model, num_classes: setattr(model, 'fc', nn.Linear(model.fc.in_features, num_classes))
    },
    'DenseNet121': {
        'constructor': models.densenet121,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Linear(model.classifier.in_features, num_classes))
    },
    'MobileNetV2': {
        'constructor': models.mobilenet_v2,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(model.last_channel, num_classes)
        ))
    },
    'EfficientNetB0': {
        'constructor': models.efficientnet_b0,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Sequential(
            model.classifier[0],
            nn.Linear(model.classifier[1].in_features, num_classes)
        ))
    }
}

# Experiment parameters
num_experiments = 50
num_epochs = 5
batch_size = 16

# Dictionary to store benchmark results and runtimes
results = {}
model_runtimes = {}

for model_name, config in model_configs.items():
    print(f"\nBenchmarking model: {model_name}")
    final_accuracies = []
    start_time = time.time()
    
    for exp in range(num_experiments):
        print(f"  Experiment {exp+1}/{num_experiments} for {model_name}")
        # Create the dataset and perform a train-test split (80% train, 20% validation)
        dataset = AgeDataset(root_dir=datafolder, transform=data_transforms)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # Instantiate the model using the constructor and modify the classification head.
        model = config['constructor'](pretrained=True)
        num_classes = len(dataset.label_map)
        config['modify'](model, num_classes)
        model = model.to(device)
        
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        
        # Training loop with validation
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            
            epoch_loss = running_loss / train_size
            val_accuracy = calculate_accuracy(model, val_loader, device)
            print(f"    Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        final_val_acc = calculate_accuracy(model, val_loader, device)
        final_accuracies.append(final_val_acc)
        print(f"  Final Val Accuracy for Experiment {exp+1}: {final_val_acc:.4f}")
    
    end_time = time.time()
    total_runtime = end_time - start_time
    model_runtimes[model_name] = total_runtime
    
    avg_acc = sum(final_accuracies) / len(final_accuracies)
    results[model_name] = avg_acc
    print(f"Average Final Val Accuracy for {model_name}: {avg_acc:.4f}")
    print(f"Total runtime for {model_name}: {total_runtime:.2f} seconds")

print("\nBenchmark Results:")
for model_name, acc in results.items():
    print(f"{model_name}: Accuracy = {acc:.4f}, Runtime = {model_runtimes[model_name]:.2f} seconds")



Benchmarking model: ResNet50
  Experiment 1/50 for ResNet50
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6567, Val Accuracy: 0.7857
    Epoch 2/5, Loss: 0.5695, Val Accuracy: 0.7857
    Epoch 3/5, Loss: 0.5296, Val Accuracy: 0.7857
    Epoch 4/5, Loss: 0.4408, Val Accuracy: 0.7857
    Epoch 5/5, Loss: 0.3909, Val Accuracy: 0.7857
  Final Val Accuracy for Experiment 1: 0.7857
  Experiment 2/50 for ResNet50
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6434, Val Accuracy: 0.5000
    Epoch 2/5, Loss: 0.5213, Val Accuracy: 0.5000
    Epoch 3/5, Loss: 0.5097, Val Accuracy: 0.5000
    Epoch 4/5, Loss: 0.3501, Val Accuracy: 0.5000
    Epoch 5/5, Loss: 0.2890, Val Accuracy: 0.5000
  Final Val Accuracy for Experiment 2: 0.5000
  Experiment 3/50 for ResNet50
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6053, Val Accuracy: 0.6429
    Epoch 2/5, Loss: 0.5458, Val Accuracy: 0.6429
    Epoch 3/5, Loss: 0.5029, Val Accuracy: 0.6429
    Epoch 4/5, Loss: 0.4169, Val Accura

/Users/hyang/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/hyang/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 57.2MB/s]


    Epoch 1/5, Loss: 0.5797, Val Accuracy: 0.6429
    Epoch 2/5, Loss: 0.5271, Val Accuracy: 0.6429
    Epoch 3/5, Loss: 0.4030, Val Accuracy: 0.6429
    Epoch 4/5, Loss: 0.3257, Val Accuracy: 0.6429
    Epoch 5/5, Loss: 0.2370, Val Accuracy: 0.6429
  Final Val Accuracy for Experiment 1: 0.6429
  Experiment 2/50 for DenseNet121
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.7942, Val Accuracy: 0.9286
    Epoch 2/5, Loss: 0.7196, Val Accuracy: 0.9286
    Epoch 3/5, Loss: 0.4246, Val Accuracy: 0.0714
    Epoch 4/5, Loss: 0.4810, Val Accuracy: 0.5000
    Epoch 5/5, Loss: 0.2930, Val Accuracy: 0.6429
  Final Val Accuracy for Experiment 2: 0.6429
  Experiment 3/50 for DenseNet121
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6873, Val Accuracy: 1.0000
    Epoch 2/5, Loss: 0.6062, Val Accuracy: 1.0000
    Epoch 3/5, Loss: 0.4391, Val Accuracy: 0.7857
    Epoch 4/5, Loss: 0.3353, Val Accuracy: 1.0000
    Epoch 5/5, Loss: 0.2964, Val Accuracy: 1.0000
  Final Val Accuracy for E

/Users/hyang/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /Users/hyang/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  Final Val Accuracy for Experiment 50: 0.7857
Average Final Val Accuracy for DenseNet121: 0.6929
Total runtime for DenseNet121: 2242.62 seconds

Benchmarking model: MobileNetV2
  Experiment 1/50 for MobileNetV2
Label mapping: {'O': 0, 'Y': 1}


100%|██████████| 13.6M/13.6M [00:00<00:00, 42.9MB/s]


    Epoch 1/5, Loss: 0.6920, Val Accuracy: 0.5714
    Epoch 2/5, Loss: 0.5211, Val Accuracy: 0.5714
    Epoch 3/5, Loss: 0.5154, Val Accuracy: 0.5714
    Epoch 4/5, Loss: 0.4016, Val Accuracy: 0.5714
    Epoch 5/5, Loss: 0.3125, Val Accuracy: 0.5714
  Final Val Accuracy for Experiment 1: 0.5714
  Experiment 2/50 for MobileNetV2
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6952, Val Accuracy: 0.7143
    Epoch 2/5, Loss: 0.5907, Val Accuracy: 0.7143
    Epoch 3/5, Loss: 0.5008, Val Accuracy: 0.7143
    Epoch 4/5, Loss: 0.4196, Val Accuracy: 0.7143
    Epoch 5/5, Loss: 0.3282, Val Accuracy: 0.7143
  Final Val Accuracy for Experiment 2: 0.7143
  Experiment 3/50 for MobileNetV2
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.7054, Val Accuracy: 0.6429
    Epoch 2/5, Loss: 0.5768, Val Accuracy: 0.6429
    Epoch 3/5, Loss: 0.5353, Val Accuracy: 0.6429
    Epoch 4/5, Loss: 0.4599, Val Accuracy: 0.5714
    Epoch 5/5, Loss: 0.3361, Val Accuracy: 0.5714
  Final Val Accuracy for E

/Users/hyang/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /Users/hyang/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 54.9MB/s]


    Epoch 1/5, Loss: 0.6717, Val Accuracy: 0.1429
    Epoch 2/5, Loss: 0.6535, Val Accuracy: 0.2143
    Epoch 3/5, Loss: 0.6192, Val Accuracy: 0.7857
    Epoch 4/5, Loss: 0.5596, Val Accuracy: 0.8571
    Epoch 5/5, Loss: 0.5330, Val Accuracy: 0.8571
  Final Val Accuracy for Experiment 1: 0.8571
  Experiment 2/50 for EfficientNetB0
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.7290, Val Accuracy: 0.3571
    Epoch 2/5, Loss: 0.6614, Val Accuracy: 0.3571
    Epoch 3/5, Loss: 0.5860, Val Accuracy: 0.5000
    Epoch 4/5, Loss: 0.5666, Val Accuracy: 0.6429
    Epoch 5/5, Loss: 0.5383, Val Accuracy: 0.6429
  Final Val Accuracy for Experiment 2: 0.6429
  Experiment 3/50 for EfficientNetB0
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.7381, Val Accuracy: 0.6429
    Epoch 2/5, Loss: 0.6720, Val Accuracy: 0.7857
    Epoch 3/5, Loss: 0.5985, Val Accuracy: 0.7143
    Epoch 4/5, Loss: 0.5683, Val Accuracy: 0.7143
    Epoch 5/5, Loss: 0.5557, Val Accuracy: 0.7143
  Final Val Accuracy

In [ ]:
import os
import time
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Custom Dataset for TIFF images
class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with all the TIFF images.
            transform (callable, optional): Transform to be applied on an image.
        """
        self.root_dir = root_dir
        self.transform = transform
        # List all TIFF files that contain 'K14'
        self.filenames = [f for f in os.listdir(root_dir) 
                          if f.lower().endswith(('.tif', '.tiff')) and ('K8' in f)]
        
        # Build label mapping using the first letter of the file name.
        # For example, if files start with 'O' or 'Y', then map them to numeric labels.
        unique_labels = sorted(set([f[0] for f in self.filenames]))
        self.label_map = {label: idx for idx, label in enumerate(unique_labels)}
        print(f"Label mapping: {self.label_map}")
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        # Get image filename and full path
        img_name = self.filenames[idx]
        img_path = os.path.join(self.root_dir, img_name)
        # Open image and convert to RGB
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        # Use the first letter of the file name as its age group label
        label_char = img_name[0]
        if label_char not in self.label_map:
            raise ValueError(f"Filename {img_name} has an invalid label character: {label_char}")
        label = self.label_map[label_char]
        return image, label

# Define data transforms (resize, tensor conversion, normalization)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (expected by most models)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Standard ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

# Path to your folder with TIFF images
datafolder = "Old_Young_Comparison"

# Function to calculate accuracy on a given DataLoader
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            # Get the predicted class (index of the highest score)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Device configuration (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dictionary with model configurations.
# Each entry contains:
# - a constructor function for the pretrained model, and
# - a modification lambda that replaces the final classifier head with one matching the dataset.
model_configs = {
    'ResNet50': {
        'constructor': models.resnet50,
        'modify': lambda model, num_classes: setattr(model, 'fc', nn.Linear(model.fc.in_features, num_classes))
    },
    'DenseNet121': {
        'constructor': models.densenet121,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Linear(model.classifier.in_features, num_classes))
    },
    'MobileNetV2': {
        'constructor': models.mobilenet_v2,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(model.last_channel, num_classes)
        ))
    },
    'EfficientNetB0': {
        'constructor': models.efficientnet_b0,
        'modify': lambda model, num_classes: setattr(model, 'classifier', nn.Sequential(
            model.classifier[0],
            nn.Linear(model.classifier[1].in_features, num_classes)
        ))
    }
}

# Experiment parameters
num_experiments = 50
num_epochs = 5
batch_size = 16

# Dictionary to store benchmark results and runtimes
results = {}
model_runtimes = {}

for model_name, config in model_configs.items():
    print(f"\nBenchmarking model: {model_name}")
    final_accuracies = []
    start_time = time.time()
    
    for exp in range(num_experiments):
        print(f"  Experiment {exp+1}/{num_experiments} for {model_name}")
        # Create the dataset and perform a train-test split (80% train, 20% validation)
        dataset = AgeDataset(root_dir=datafolder, transform=data_transforms)
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # Instantiate the model using the constructor and modify the classification head.
        model = config['constructor'](pretrained=True)
        num_classes = len(dataset.label_map)
        config['modify'](model, num_classes)
        model = model.to(device)
        
        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        
        # Training loop with validation
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            
            epoch_loss = running_loss / train_size
            val_accuracy = calculate_accuracy(model, val_loader, device)
            print(f"    Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        final_val_acc = calculate_accuracy(model, val_loader, device)
        final_accuracies.append(final_val_acc)
        print(f"  Final Val Accuracy for Experiment {exp+1}: {final_val_acc:.4f}")
    
    end_time = time.time()
    total_runtime = end_time - start_time
    model_runtimes[model_name] = total_runtime
    
    avg_acc = sum(final_accuracies) / len(final_accuracies)
    results[model_name] = avg_acc
    print(f"Average Final Val Accuracy for {model_name}: {avg_acc:.4f}")
    print(f"Total runtime for {model_name}: {total_runtime:.2f} seconds")

print("\nBenchmark Results:")
for model_name, acc in results.items():
    print(f"{model_name}: Accuracy = {acc:.4f}, Runtime = {model_runtimes[model_name]:.2f} seconds")



Benchmarking model: DenseNet121
  Experiment 1/16 for DenseNet121
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6507, Val Accuracy: 0.7857
    Epoch 2/5, Loss: 0.5338, Val Accuracy: 0.7857
    Epoch 3/5, Loss: 0.4325, Val Accuracy: 0.7857
    Epoch 4/5, Loss: 0.3530, Val Accuracy: 0.7857
    Epoch 5/5, Loss: 0.2634, Val Accuracy: 0.7857
  Final Val Accuracy for Experiment 1: 0.7857
  Experiment 2/16 for DenseNet121
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.6498, Val Accuracy: 0.6429
    Epoch 2/5, Loss: 0.5260, Val Accuracy: 0.6429
    Epoch 3/5, Loss: 0.3965, Val Accuracy: 0.6429
    Epoch 4/5, Loss: 0.3081, Val Accuracy: 0.6429
    Epoch 5/5, Loss: 0.2534, Val Accuracy: 0.6429
  Final Val Accuracy for Experiment 2: 0.6429
  Experiment 3/16 for DenseNet121
Label mapping: {'O': 0, 'Y': 1}
    Epoch 1/5, Loss: 0.7458, Val Accuracy: 0.5714
    Epoch 2/5, Loss: 0.6297, Val Accuracy: 0.5714
    Epoch 3/5, Loss: 0.4751, Val Accuracy: 0.6429
    Epoch 4/5, Loss: 0.3156